In [1]:
from sklearn.metrics import confusion_matrix
import datetime as dt
import numpy as np
import pandas as pd

path = ""

In [2]:
#load the labels
test_label = [i.strip() for i in open(path+"test-label.dat").readlines()]
test_label = np.array(test_label)
train_label = [i.strip() for i in open(path+"train-label.dat").readlines()]
train_label = np.array(train_label)

#load the data 3 times (for each classifier,)
dfKNN = pd.read_csv("train_kmeans_svd.csv") 
df_testKNN = pd.read_csv("test_kmeans_svd.csv")
dfSVM = pd.read_csv("train_kmeans_svd.csv") 
df_testSVM = pd.read_csv("test_kmeans_svd.csv")
dfRF= pd.read_csv("train_kmeans_svd.csv") 
df_testRF = pd.read_csv("test_kmeans_svd.csv")

#shape the labels by label (each row represents one label) 
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels = train_labels.transpose()

#test labels
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels = test_labels.transpose()


#normalize the data
from sklearn import preprocessing
x1 = dfKNN.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled1 = min_max_scaler.fit_transform(x1)
dfKNN = pd.DataFrame(x_scaled1)
x1 = df_testKNN.values #returns a numpy array
x_scaled1 = min_max_scaler.transform(x1)
df_testKNN = pd.DataFrame(x_scaled1)
dfinitKNN = dfKNN.copy()



x2 = dfSVM.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled2 = min_max_scaler.fit_transform(x2)
dfSVM = pd.DataFrame(x_scaled2)
x2 = df_testSVM.values #returns a numpy array
x_scaled2 = min_max_scaler.transform(x2)
df_testSVM = pd.DataFrame(x_scaled2)
dfinitSVM = dfSVM.copy()


x3 = dfRF.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled3 = min_max_scaler.fit_transform(x3)
dfRF = pd.DataFrame(x_scaled3)
x3 = df_testRF.values #returns a numpy array
x_scaled3 = min_max_scaler.transform(x3)
df_testRF = pd.DataFrame(x_scaled3)
dfinitRF = dfRF.copy()



In [3]:
def multilabel_confusion_matrix(predict,path,clustnum=-3):
    test_label = [i.strip().split() for i in open(path + "test-label.dat").readlines()]

    for i in range(len(test_label)):
        item = test_label[i]
        for j in range(len(item)):
            item[j] = int(item[j])

        test_label[i] = item

    labels = []

    for item in open(path + "labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(test_label)
    y_pred = np.array(predict)

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)
    f = open("confmatrix"+str(clustnum)+".txt", "w")
    f.close()
    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        f = open("confmatrix"+str(clustnum)+".txt", "a")
        print(matrix)
        f.write("Confusion matrix for label {}:".format(label))
        np.savetxt(f,matrix)
        f.close()
    return

In [4]:
# apply a combination of oversampling and undersampling technique
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
sm = SMOTE(sampling_strategy=0.5, random_state=2)
smsmall = SMOTE(sampling_strategy=0.25, random_state=2)
rm = RandomUnderSampler(random_state=42)


# Fit an ensemble of 10 classifier chains and take the average prediction of all the chains.

orderlist1 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
#orderlist3 = [1,3,15,5,7,9,11,2,4,13,6,18,8,17,10,14,12,19,16,0]
#orderlist4 = [2,4,17,6,8,3,13,19,9,10,14,1,12,0,16,5,7,11,15,18]
#orderlist5 = [19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
#orderlist6 = [4,6,3,15,11,7,10,1,13,5,0,12,16,2,18,9,17,8,14,19]
#orderlist7 = [15,6,8,0,3,13,18,10,12,1,4,2,17,14,16,11,9,7,19,5]
#orderlist8 = [12,7,4,14,16,6,0,19,10,8,3,5,1,9,2,15,18,11,13,17]
#orderlist9 = [5,8,19,10,6,12,16,0,3,11,14,4,7,9,17,15,18,13,2,1]
#orderlist10 =[19,18,1,16,13,3,14,15,11,17,9,10,12,5,6,8,0,4,7,2]
OrderList = [orderlist1]#, orderlist2, orderlist3, orderlist4, orderlist5, orderlist6, orderlist7, orderlist8, orderlist9, orderlist10]


# this function ordering the predictions of every chain in the same way
# to fit the confusion matrix correctly
def ordering_labels(order,predictions):
    new_predictions = []
    for pred in predictions:
        temp_predict = [0] * len(pred) 
        for i in range(0,len(pred)):
            temp_predict[ order[i] ] = pred[i]
        new_predictions.append(temp_predict)
    return new_predictions   

In [5]:
###############################################################################
#.................................KNN..........................................
###############################################################################
from sklearn import neighbors
predKNN = pd.DataFrame()
KNNpredictions= []
start_time = dt.datetime.now()
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfKNN, train_labels_new = smsmall.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfKNN, train_labels_new = sm.fit_resample(dfinitKNN,train_labels[:][l])
            dfKNN, train_labels_new = rm.fit_resample(dfKNN,train_labels_new)
        else:
            dfKNN, train_labels_new = rm.fit_resample(dfinitKNN, train_labels[:][l])
        #print(dfKNN.shape)
        KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1).fit(dfKNN, train_labels_new)
        dfinitKNN['Label'+str(l)] = train_labels[:][l]
        df_testKNN['Label'+str(l)] = KNN.predict(df_testKNN) 
        predKNN['Label'+str(l)] = df_testKNN['Label'+str(l)]


    
    predictKNN=[]
    for p in range(3983):
        #print(p)
        resultKNN = []
        for r in range(20):
            x = orderlist.index(r)
            resultKNN.append(predKNN.iloc[p,x])
        predictKNN.append(resultKNN)
    KNNpredictions.append(predictKNN)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)

the training time is: 0:00:12.830612


In [6]:
final_predictionsKNN = []
for i in range(0,len(KNNpredictions)):
    final_predictionsKNN.append(ordering_labels( OrderList[i], KNNpredictions[i]))


In [7]:
FinalKNN = []
for i in range(0,len(final_predictionsKNN[0])):
    tempKNN = [0] * len(final_predictionsKNN[0][0])
    for j in range(0,len(final_predictionsKNN)):
        tempKNN = [tempKNN[k] + final_predictionsKNN[j][i][k] for k in range(len(tempKNN))]
    tempKNN = [int(x / len(final_predictionsKNN) + 0.5) for x in tempKNN]
    FinalKNN.append(tempKNN)
    

In [8]:
#...........KNN..................
#confusion matrix for every label
print('for KNN classifier:')
multilabel_confusion_matrix(FinalKNN,path)


for KNN classifier:
Confusion matrix for label programming:
[[2003 1003]
 [ 438  539]]
Confusion matrix for label style:
[[2913  842]
 [ 176   52]]
Confusion matrix for label reference:
[[1393 1032]
 [ 810  748]]
Confusion matrix for label java:
[[2553 1058]
 [ 205  167]]
Confusion matrix for label web:
[[1886 1047]
 [ 482  568]]
Confusion matrix for label internet:
[[2701  745]
 [ 386  151]]
Confusion matrix for label culture:
[[2410  871]
 [ 422  280]]
Confusion matrix for label design:
[[1402 1502]
 [ 469  610]]
Confusion matrix for label education:
[[2537  643]
 [ 603  200]]
Confusion matrix for label language:
[[2770  730]
 [ 379  104]]
Confusion matrix for label books:
[[2667  809]
 [ 366  141]]
Confusion matrix for label writing:
[[2687  818]
 [ 349  129]]
Confusion matrix for label computer:
[[2828  646]
 [ 388  121]]
Confusion matrix for label english:
[[2938  690]
 [ 277   78]]
Confusion matrix for label politics:
[[3081  510]
 [ 305   87]]
Confusion matrix for label history:

In [9]:
#.......KNN.......
#Balanced Accuracy
FinalKNN=pd.DataFrame(FinalKNN)
from sklearn.metrics import balanced_accuracy_score
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print('for kNN classifier')
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalKNN[i]))
    
   


for kNN classifier
the Balanced Accuracy for the labels 
programming : 0.6090114210337427
style : 0.5019179106220946
reference : 0.5272678427272607
java : 0.5779655503047736
web : 0.5919899988635072
internet : 0.5324995595789683
culture : 0.5666962768456215
design : 0.5240603226638276
education : 0.5234323723145125
language : 0.5033747412008281
books : 0.5226838643343025
writing : 0.5182467962683315
computer : 0.5258841147202966
english : 0.5147654393838222
politics : 0.5399585272705574
history : 0.5533126293995859
philosophy : 0.5610485193170421
science : 0.504043320596028
religion : 0.5431590569475149
grammar : 0.49898154477101847


In [10]:
###############################################################################
#.................................SVM..........................................
###############################################################################

from sklearn import svm
predSVM = pd.DataFrame()

SVMpredictions= []
start_time = dt.datetime.now()

for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfSVM, train_labels_new2 = smsmall.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfSVM, train_labels_new2 = sm.fit_resample(dfinitSVM,train_labels[:][l])
            dfSVM, train_labels_new2 = rm.fit_resample(dfSVM,train_labels_new2)
        else:
            dfSVM, train_labels_new2 = rm.fit_resample(dfinitSVM, train_labels[:][l])
        #print(df.shape)
        SVM= svm.SVC(C=10 ,gamma=1.5, kernel='rbf' , random_state=41).fit(dfSVM, train_labels_new2)
        dfinitSVM['Label'+str(l)] = train_labels[:][l]
        df_testSVM['Label'+str(l)] = SVM.predict(df_testSVM) 
        predSVM['Label'+str(l)] = df_testSVM['Label'+str(l)]


    
    predictSVM=[]
    for p in range(3983):
        #print(p)
        resultSVM = []
        for r in range(20):
            x = orderlist.index(r)
            resultSVM.append(predSVM.iloc[p,x])
        predictSVM.append(resultSVM)
    SVMpredictions.append(predictSVM)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)

the training time is: 0:00:34.787948


In [11]:
final_predictionsSVM = []
for i in range(0,len(SVMpredictions)):
    final_predictionsSVM.append(ordering_labels( OrderList[i], SVMpredictions[i]))
    
FinalSVM = []
for i in range(0,len(final_predictionsSVM[0])):
    tempSVM = [0] * len(final_predictionsSVM[0][0])
    for j in range(0,len(final_predictionsSVM)):
        tempSVM = [tempSVM[k] + final_predictionsSVM[j][i][k] for k in range(len(tempSVM))]
    tempSVM = [int(x / len(final_predictionsSVM) + 0.5) for x in tempSVM]
    FinalSVM.append(tempSVM)
    

In [12]:
#...........SVM..................
#confusion matrix for every label
print("for SVM classifier:")
multilabel_confusion_matrix(FinalSVM,path)


for SVM classifier:
Confusion matrix for label programming:
[[2076  930]
 [ 345  632]]
Confusion matrix for label style:
[[2201 1554]
 [ 110  118]]
Confusion matrix for label reference:
[[1591  834]
 [ 906  652]]
Confusion matrix for label java:
[[2539 1072]
 [ 174  198]]
Confusion matrix for label web:
[[2195  738]
 [ 554  496]]
Confusion matrix for label internet:
[[2995  451]
 [ 435  102]]
Confusion matrix for label culture:
[[1734 1547]
 [ 209  493]]
Confusion matrix for label design:
[[ 907 1997]
 [ 304  775]]
Confusion matrix for label education:
[[2841  339]
 [ 717   86]]
Confusion matrix for label language:
[[3113  387]
 [ 422   61]]
Confusion matrix for label books:
[[2880  596]
 [ 390  117]]
Confusion matrix for label writing:
[[2936  569]
 [ 400   78]]
Confusion matrix for label computer:
[[2979  495]
 [ 412   97]]
Confusion matrix for label english:
[[3390  238]
 [ 334   21]]
Confusion matrix for label politics:
[[3217  374]
 [ 306   86]]
Confusion matrix for label history:

In [13]:
#.......SVM.......
#Balanced Accuracy
FinalSVM=pd.DataFrame(FinalSVM)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalSVM[i]))
    

the Balanced Accuracy for the labels 
programming : 0.668748480521046
style : 0.5518478286261592
reference : 0.537283855855379
java : 0.6176936957861731
web : 0.6103807250823957
internet : 0.5295338778342309
culture : 0.6153883058028136
design : 0.5152927348299747
education : 0.500247303743039
language : 0.5078612836438924
books : 0.5296538904133841
writing : 0.5004202006696948
computer : 0.5240413489825626
english : 0.4967770237743994
politics : 0.557619246529021
history : 0.528559777006982
philosophy : 0.5589285392556648
science : 0.5155808429092419
religion : 0.5083012210349628
grammar : 0.4988174982911825


In [14]:
###############################################################################
#..........................Random Forest.....................................
###############################################################################
from sklearn.ensemble import RandomForestClassifier
predRF = pd.DataFrame()
RFpredictions= []
start_time = dt.datetime.now()

      
      
for orderlist in OrderList:
    for l in orderlist:
        #print(train_labels[train_labels[:][l] == 1][l].size)
        if train_labels[train_labels[:][l] == 1][l].size < 1000:
            #print("kkk")
            dfRF, train_labels_new3 = smsmall.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        elif train_labels[train_labels[:][l] == 1][l].size < 2000:
            dfRF, train_labels_new3 = sm.fit_resample(dfinitRF,train_labels[:][l])
            dfRF, train_labels_new3 = rm.fit_resample(dfRF,train_labels_new3)
        else:
            dfRF, train_labels_new3 = rm.fit_resample(dfinitRF, train_labels[:][l])
        #print(df.shape)
        Random_Forest = RandomForestClassifier(n_estimators=20, criterion='gini', random_state=41).fit(dfRF, train_labels_new3)
        dfinitRF['Label'+str(l)] = train_labels[:][l]
        df_testRF['Label'+str(l)] = Random_Forest.predict(df_testRF) 
        predRF['Label'+str(l)] = df_testRF['Label'+str(l)]


    
    predictRF=[]
    for p in range(3983):
        #print(p)
        resultRF = []
        for r in range(20):
            x = orderlist.index(r)
            resultRF.append(predRF.iloc[p,x])
        predictRF.append(resultRF)
    RFpredictions.append(predictRF)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)  

the training time is: 0:00:03.796826


In [15]:
final_predictionsRF = []
for i in range(0,len(RFpredictions)):
    final_predictionsRF.append(ordering_labels( OrderList[i], RFpredictions[i]))
    
FinalRF = []
for i in range(0,len(final_predictionsRF[0])):
    tempRF = [0] * len(final_predictionsRF[0][0])
    for j in range(0,len(final_predictionsRF)):
        tempRF = [tempRF[k] + final_predictionsRF[j][i][k] for k in range(len(tempRF))]
    tempRF = [int(x / len(final_predictionsRF) + 0.5) for x in tempRF]
    FinalRF.append(tempRF)
    

In [16]:
#........Random Forest...........
#confusion matrix for every label
print("for Random Forest classifier:")
multilabel_confusion_matrix(FinalRF,path)

for Random Forest classifier:
Confusion matrix for label programming:
[[1960 1046]
 [ 370  607]]
Confusion matrix for label style:
[[3525  230]
 [ 203   25]]
Confusion matrix for label reference:
[[1388 1037]
 [ 753  805]]
Confusion matrix for label java:
[[2579 1032]
 [ 197  175]]
Confusion matrix for label web:
[[1887 1046]
 [ 404  646]]
Confusion matrix for label internet:
[[2958  488]
 [ 416  121]]
Confusion matrix for label culture:
[[2762  519]
 [ 497  205]]
Confusion matrix for label design:
[[1558 1346]
 [ 507  572]]
Confusion matrix for label education:
[[2852  328]
 [ 676  127]]
Confusion matrix for label language:
[[3252  248]
 [ 430   53]]
Confusion matrix for label books:
[[3249  227]
 [ 452   55]]
Confusion matrix for label writing:
[[3138  367]
 [ 404   74]]
Confusion matrix for label computer:
[[3201  273]
 [ 455   54]]
Confusion matrix for label english:
[[3346  282]
 [ 313   42]]
Confusion matrix for label politics:
[[3000  591]
 [ 258  134]]
Confusion matrix for labe

In [17]:
#..Random Forest..
#Balanced Accuracy
FinalRF=pd.DataFrame(FinalRF)
labels=['programming','style', 'reference', 'java','web', 'internet', 'culture', 'design', 'education', 'language', 'books', 'writing', 'computer','english', 'politics','history','philosophy', 'science','religion', 'grammar']
print("the Balanced Accuracy for the labels ")
for i in range (0,20):
    print(labels[i],":",balanced_accuracy_score(test_labels[i], FinalRF[i]))
    

the Balanced Accuracy for the labels 
programming : 0.6366594685075431
style : 0.524198729179807
reference : 0.5445295978190384
java : 0.5923183492494558
web : 0.6293033299238551
internet : 0.5418562098284682
culture : 0.5669196556883238
design : 0.5333109296690897
education : 0.5275061287467594
language : 0.5194368530020704
books : 0.5215881570555378
writing : 0.5250520774267484
computer : 0.5137533040843403
english : 0.5202905414848518
politics : 0.5886293113736722
history : 0.5667522608516398
philosophy : 0.6082896425261194
science : 0.523902510888619
religion : 0.5681081378449193
grammar : 0.5103349282296651
